In [1]:
import pyarrow as pa
import pyarrow.fs
import requests
import io
import os

In [2]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 34283392124 (31.93 GB)
DFS Remaining: 33754406912 (31.44 GB)
DFS Used: 528985212 (504.48 MB)
DFS Used%: 1.54%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.4:9866 (project-4-batch04-dn-2.project-4-batch04_default)
Hostname: 4238186f1c34
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 272503121 (259.88 MB)
Non DFS Used: 8654569135 (8.06 GB)
DFS Remaining: 16877203456 (15.72 GB)
DFS Used%: 1.06%
DFS Remaining

In [3]:
if not os.path.exists("hdma-wi-2021.csv"):
    !wget "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"
    
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

Deleted hdfs://boss:9000/single.csv


In [4]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/double.csv
!hdfs dfs -du -h hdfs://boss:9000/single.csv

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


In [5]:
#q3
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
r.raise_for_status()
single_status = r.json()
single_status

{'FileStatus': {'accessTime': 1698094454438,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16388,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1698094458897,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [6]:
#q4
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.raise_for_status()
r.json()

{'Location': 'http://332f988c4bb3:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'}

In [7]:
#q5
offset = 0
length = single_status['FileStatus']['length']
blockSize = single_status['FileStatus']['blockSize']

blockDict = {}

while offset < length:
    r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&noredirect=true")
    r.raise_for_status()
    id = r.json().get("Location").split(":")[1][2:]
    if id in blockDict:
        blockDict[id] += 1
    else:
        blockDict[id] = 1
    offset += blockSize
    
blockDict

{'332f988c4bb3': 92, '4238186f1c34': 75}

In [8]:
#q6
hdfs = pa.fs.HadoopFileSystem("boss", 9000)
f = hdfs.open_input_file("/single.csv")
f.read_at(10,0)

2023-10-23 20:54:43,166 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [9]:
#q7
count = 0
with hdfs.open_input_file("/single.csv") as f:
    reader = io.TextIOWrapper(io.BufferedReader(f))
    for i, line in enumerate(reader):
        if "Single Family" in line:
            count += 1
count

444874